# TinyAleph: ResoFormer - Machine Learning Architecture

This notebook explores the ML components:

- **SparsePrimeState**: Quaternionic amplitudes
- **Tensor operations**: Pure Python implementation
- **Layers**: Dense, Attention, Gating, Collapse
- **ResoFormer**: Complete transformer architecture

In [ ]:
import sys
sys.path.insert(0, '..')

from tinyaleph.ml.sparse_state import (
    SparsePrimeState, coherent_superposition, golden_superposition
)
from tinyaleph.ml.resoformer import (
    Tensor, zeros, ones, randn, glorot_uniform,
    Dense, QuaternionDense, LayerNorm, Dropout,
    ResonantAttentionLayer, CoherenceGatingLayer, EntropyCollapseLayer,
    ResoFormerBlock, ResoFormerConfig,
    create_resoformer_model, create_resoformer_classifier
)
from tinyaleph.core.constants import PHI
import math

print("TinyAleph ML - ResoFormer")

## 1. Sparse Prime States

States in H_Q = H_P ⊗ ℍ (Prime-Quaternion tensor product)

In [ ]:
# Create sparse state
state = SparsePrimeState.from_primes([2, 3, 5, 7])

print(f"SparsePrimeState with {len(state)} amplitudes")
print(f"\nQuaternion amplitudes:")
for prime, q in state:
    print(f"  |{prime}⟩: ({q.w:.3f}, {q.i:.3f}i, {q.j:.3f}j, {q.k:.3f}k)")

In [ ]:
# Properties
print(f"Entropy: {state.entropy():.4f}")
print(f"Is coherent: {state.is_coherent()}")
print(f"\nPrime spectrum:")
for p, prob in state.prime_spectrum().items():
    print(f"  P({p}) = {prob:.4f}")

In [ ]:
# Golden superposition - uses golden angle spacing
golden = golden_superposition(5)

print("Golden superposition (2π/φ² ≈ 137.5° spacing):")
for p, q in golden:
    print(f"  |{p}⟩: ({q.w:.3f}, {q.i:.3f}i)")

## 2. Tensor Operations

In [ ]:
# Create tensors
z = zeros((3, 4))
o = ones((2, 3))
r = randn((4, 4))
g = glorot_uniform((32, 64))

print(f"zeros((3, 4)): shape = {z.shape}")
print(f"ones((2, 3)): shape = {o.shape}")
print(f"randn((4, 4)): shape = {r.shape}")
print(f"glorot_uniform((32, 64)): shape = {g.shape}")

In [ ]:
# Tensor operations
a = Tensor([1.0, 2.0, 3.0, 4.0])
b = Tensor([0.5, 0.5, 0.5, 0.5])

print(f"a = {a.data}")
print(f"b = {b.data}")
print(f"\na + b = {(a + b).data}")
print(f"a * b = {(a * b).data}")
print(f"a.sum() = {a.sum().data}")
print(f"a.mean() = {a.mean().data}")

In [ ]:
# Activation functions
x = Tensor([-2.0, -1.0, 0.0, 1.0, 2.0])

print(f"x = {x.data}")
print(f"relu(x) = {x.relu().data}")
print(f"sigmoid(x) = {[round(v, 3) for v in x.sigmoid().data]}")
print(f"tanh(x) = {[round(v, 3) for v in x.tanh().data]}")

## 3. Dense Layers

In [ ]:
# Standard dense layer
dense = Dense(units=16, activation="relu", name="dense1")

x = randn((4, 8))  # batch=4, input_dim=8
y = dense(x)

print(f"Dense layer:")
print(f"  Input: {x.shape}")
print(f"  Output: {y.shape}")

In [ ]:
# Quaternion dense layer
q_dense = QuaternionDense(units=8, name="q_dense1")

x_q = randn((4, 32))
y_q = q_dense(x_q)

print(f"QuaternionDense layer:")
print(f"  Input: {x_q.shape}")
print(f"  Output: {y_q.shape} (units × 4 quaternion components)")

## 4. Attention Layer

In [ ]:
# Resonant attention
attn = ResonantAttentionLayer(num_heads=4, key_dim=16, name="attn1")

x = randn((2, 8, 64))  # batch=2, seq=8, embed=64
y = attn(x)

print(f"ResonantAttentionLayer (4 heads, key_dim=16):")
print(f"  Input: {x.shape}")
print(f"  Output: {y.shape}")

## 5. Special Layers

In [ ]:
# Coherence gating
gating = CoherenceGatingLayer(threshold=1/PHI, name="gate1")

x = randn((4, 32))
result = gating(x)

print(f"CoherenceGatingLayer (threshold=1/φ):")
print(f"  Input: {x.shape}")
print(f"  Output: {result['output'].shape}")
print(f"  Coherence: {result['coherence'].shape}")
print(f"  Gate: {result['gate'].shape}")

In [ ]:
# Entropy collapse (VQ-style)
collapse = EntropyCollapseLayer(num_attractors=16, name="collapse1")

x = randn((4, 32))
result = collapse(x, training=True)

print(f"EntropyCollapseLayer (16 attractors):")
print(f"  Input: {x.shape}")
print(f"  Output: {result['output'].shape}")
print(f"  Probs: {result['probs'].shape}")
print(f"  Entropy: {[round(e, 3) for e in result['entropy'].data]}")

## 6. ResoFormer Block

In [ ]:
# Complete transformer block
block = ResoFormerBlock(
    dim=64,
    num_heads=4,
    ffn_dim=256,
    dropout_rate=0.1,
    use_collapse=True,
    name="block1"
)

print("ResoFormerBlock architecture:")
print("  1. LayerNorm → Attention → Residual")
print("  2. ResonanceOperator")
print("  3. LayerNorm → FFN → Residual")
print("  4. CoherenceGating")
print("  5. EntropyCollapse")

In [ ]:
# Forward pass
x = randn((2, 8, 64))  # batch=2, seq=8, dim=64
y = block(x, training=True)

print(f"\nForward pass:")
print(f"  Input: {x.shape}")
print(f"  Output: {y.shape}")

## 7. ResoFormer Model

In [ ]:
# Configuration
config = ResoFormerConfig(
    vocab_size=1000,
    seq_len=32,
    dim=64,
    num_layers=2,
    num_heads=4,
    ffn_dim=128,
    dropout=0.1,
)

print("ResoFormerConfig:")
print(f"  vocab_size: {config.vocab_size}")
print(f"  seq_len: {config.seq_len}")
print(f"  dim: {config.dim}")
print(f"  num_layers: {config.num_layers}")
print(f"  num_heads: {config.num_heads}")

In [ ]:
# Language model
lm = create_resoformer_model(
    vocab_size=1000,
    seq_len=32,
    dim=64,
    num_layers=2,
    num_heads=4,
    ffn_dim=128,
)

tokens = Tensor([list(range(8))], (1, 8))  # batch=1, seq=8
logits = lm(tokens, training=False)

print(f"Language Model:")
print(f"  Input tokens: {tokens.shape}")
print(f"  Output logits: {logits.shape}")

In [ ]:
# Classifier
clf = create_resoformer_classifier(
    vocab_size=1000,
    seq_len=32,
    dim=64,
    num_layers=2,
    num_heads=4,
    ffn_dim=128,
    num_classes=10,
)

output = clf(tokens, training=False)

print(f"Classifier:")
print(f"  Input: {tokens.shape}")
print(f"  Output: {output.shape} (10 classes)")

## Summary

**SparsePrimeState:**
- H_Q = H_P ⊗ ℍ (Prime-Quaternion tensor)
- Quaternion amplitudes per prime
- Entropy and coherence metrics

**Layers:**
- Dense and QuaternionDense
- ResonantAttentionLayer (multi-head)
- CoherenceGatingLayer (threshold-based)
- EntropyCollapseLayer (VQ-style)

**ResoFormer:**
- Pre-norm transformer blocks
- Resonance operators for phase rotation
- Coherence-gated computation
- Model variants: LM, Classifier, Embedder